In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip list

Package               Version
--------------------- -----------
argon2-cffi           21.3.0
argon2-cffi-bindings  21.2.0
asttokens             2.0.8
attrs                 22.1.0
backcall              0.2.0
beautifulsoup4        4.11.1
bleach                5.0.1
certifi               2022.6.15
cffi                  1.15.1
charset-normalizer    2.1.1
colorama              0.4.5
cycler                0.11.0
debugpy               1.6.3
decorator             5.1.1
defusedxml            0.7.1
entrypoints           0.4
executing             0.10.0
fastjsonschema        2.16.1
fonttools             4.37.0
htmlmin               0.1.12
idna                  3.3
ImageHash             4.2.1
importlib-metadata    4.12.0
ipykernel             6.15.1
ipython               8.4.0
ipython-genutils      0.2.0
ipywidgets            8.0.1
jedi                  0.18.1
Jinja2                3.1.2
joblib                1.1.0
jsonschema            4.14.0
jupyter-client        7.3.4
jupyter-core          4.11


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Functions

In [3]:
%%writefile ./scripts/python/get_metrics.py

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

# POST PROCESSING BIAS METRICS FOR THE TRAINED MODEL

def get_PPL(data_frame, column_to_group_by, column_to_count):
    """
    :param data_frame: 
    :param column_to_group_by: 
    :param column_to_count: 
    :return: 
    """
    
    ppl = []
    
    counts_total = data_frame.groupby([column_to_group_by]) \
    .size() \
    .rename('count_total') \
    .reset_index() \
    .sort_values(by=column_to_group_by, ascending=False) \
    .set_index(column_to_group_by)
    
    
    counts_grouped = data_frame.groupby([column_to_group_by, column_to_count]) \
    .size() \
    .rename('count') \
    .reset_index() \
    .sort_values(by=column_to_group_by, ascending=False) \
    .set_index(column_to_group_by)

    positive_counts = counts_grouped.loc[counts_grouped[column_to_count] == 1]
    df = pd.merge(positive_counts, counts_total, left_index=True, right_index=True)
    df['PPL'] = df['count'] / df['count_total']
    
    return df['PPL']


def get_DPPL(data_frame, column_to_group_by, column_to_count):
    """
    :param data_frame: 
    :param column_to_group_by: 
    :param column_to_count: 
    :return: 
    """
    
    ppl = []
    
    df = get_PPL(data_frame, column_to_group_by, column_to_count)

    for i in df.iteritems():
        ppl.append(i[1])
        
    return sorted(set([i - j for i in ppl for j in ppl if i != j]))


def get_CA(data_frame, column_to_group_by, column_to_count_acctuals, column_to_count_predictions):
    """
    :param data_frame: 
    :param column_to_group_by: 
    :param column_to_count_acctuals: 
    :param column_to_count_predictions: 
    :return: 
    """
    
    counts_total_labels = data_frame.groupby([column_to_group_by]) \
    .size() \
    .rename('count_total') \
    .reset_index() \
    .sort_values(by=column_to_group_by, ascending=False) \
    .set_index(column_to_group_by)
    
    
    counts_grouped_acctuals = data_frame.groupby([column_to_group_by, column_to_count_acctuals]) \
    .size() \
    .rename('count_grouped_acctuals') \
    .reset_index() \
    .sort_values(by=column_to_group_by, ascending=False) \
    .set_index(column_to_group_by)
    
    positive_counts_grouped_acctuals = counts_grouped_acctuals.loc[counts_grouped_acctuals[column_to_count_acctuals] == 1]
    
    
    counts_grouped_predictions = data_frame.groupby([column_to_group_by, column_to_count_predictions]) \
    .size() \
    .rename('count_gruped_predictions') \
    .reset_index() \
    .sort_values(by=column_to_group_by, ascending=False) \
    .set_index(column_to_group_by)

    positive_counts_grouped_predictions= counts_grouped_predictions.loc[counts_grouped_predictions[column_to_count_predictions] == 1]


    df = pd.merge(positive_counts_grouped_acctuals, positive_counts_grouped_predictions, left_index=True, right_index=True)
    df['CA'] = df['count_grouped_acctuals'] / df['count_gruped_predictions']
            
    return df['CA']


def get_DCA(data_frame, column_to_group_by, column_to_count_acctuals, column_to_count_predictions):
    """
    :param data_frame: 
    :param column_to_group_by: 
    :param column_to_count_acctuals: 
    :param column_to_count_predictions: 
    :return: 
    """
    
    ca = []
    
    df = get_CA(data_frame, column_to_group_by, column_to_count_acctuals, column_to_count_predictions)
    
    for i in df.iteritems():
        ca.append(i[1])
            
    return sorted(set([i - j for i in ca for j in ca if i != j]))


def get_CR(data_frame, column_to_group_by, column_to_count_acctuals, column_to_count_predictions):
    """
    :param data_frame: 
    :param column_to_group_by: 
    :param column_to_count_acctuals: 
    :param column_to_count_predictions: 
    :return: 
    """
    
    counts_total_labels = data_frame.groupby([column_to_group_by]) \
    .size() \
    .rename('count_total') \
    .reset_index() \
    .sort_values(by=column_to_group_by, ascending=False) \
    .set_index(column_to_group_by)
    
    
    counts_grouped_acctuals = data_frame.groupby([column_to_group_by, column_to_count_acctuals]) \
    .size() \
    .rename('count_grouped_acctuals') \
    .reset_index() \
    .sort_values(by=column_to_group_by, ascending=False) \
    .set_index(column_to_group_by)
    
    negative_counts_grouped_acctuals = counts_grouped_acctuals.loc[counts_grouped_acctuals[column_to_count_acctuals] == 0]
    
    
    counts_grouped_predictions = data_frame.groupby([column_to_group_by, column_to_count_predictions]) \
    .size() \
    .rename('count_gruped_predictions') \
    .reset_index() \
    .sort_values(by=column_to_group_by, ascending=False) \
    .set_index(column_to_group_by)

    negative_counts_grouped_predictions= counts_grouped_predictions.loc[counts_grouped_predictions[column_to_count_predictions] == 0]


    df = pd.merge(negative_counts_grouped_acctuals, negative_counts_grouped_predictions, left_index=True, right_index=True)
    df['CR'] = df['count_grouped_acctuals'] / df['count_gruped_predictions']
            
    return df['CR']


def get_DCR(data_frame, column_to_group_by, column_to_count_acctuals, column_to_count_predictions):
    """
    :param data_frame: 
    :param column_to_group_by: 
    :param column_to_count_acctuals: 
    :param column_to_count_predictions: 
    :return: 
    """
    
    cr = []
    
    df = get_CR(data_frame, column_to_group_by, column_to_count_acctuals, column_to_count_predictions)
    
    for i in df.iteritems():
        cr.append(i[1])
    
    return sorted(set([i - j for i in cr for j in cr if i != j]))


def get_cm(data_frame, acctuals, predictions):
    
    cm = confusion_matrix(data_frame[acctuals], data_frame[predictions].notnull())
    TN, FN, FP, TP = cm[0][0], cm[1][0], cm[0][1], cm[1][1]
    
    return TN, FN, FP, TP


def get_class_cm(data_frame, acctuals, predictions, column_to_group_by):
    
    data = pd.DataFrame(columns=['Class', 'TN', 'FN', 'FP', 'TP'])
    
    for group in data_frame[column_to_group_by].dropna().unique():
        sample = data_frame.loc[data_frame[column_to_group_by] == group]
        cm = confusion_matrix(sample[acctuals], sample[predictions].notnull())
        try:
            TN, FN, FP, TP = cm[0][0], cm[1][0], cm[0][1], cm[1][1]
        except:
            pass
        else:
            TN, FN, FP, TP = cm[0][0], cm[1][0], cm[0][1], cm[1][1]
        
        data = data.append({'Class': group, 'TN': TN, 'FN': FN, 'FP': FP, 'TP': TP}, ignore_index=True)
        
    return data


def get_RD(data_frame, acctuals, predictions, column_to_group_by):
    
    r = []
    
    data = get_class_cm(data_frame, acctuals, predictions, column_to_group_by)
    
    try:
        data['Recall'] = (data['TP'] / (data['TP'] + data['FN']))
    except ZeroDivisionError:
        data['Recall'] = 0
           
    for i in data['Recall'].iteritems():
        r.append(i[1])
    
    return sorted(set([i - j for i in r for j in r if i != j]))


Overwriting ./scripts/python/get_metrics.py


# Libraries

In [4]:
# import sys
# print(sys.path)
# sys.path.insert(0, "C:\\Users\\KonuTech\\gfk_data_analyst\\gfk_ml_ops_data_analyst\\scripts\\python\\")

In [5]:
import os
# import sys
import json
import numpy as np
import pandas as pd
# import math
from pandas_profiling import ProfileReport
# import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from scripts.python.get_metrics import get_PPL, get_DPPL, get_CA, get_DCA, get_CR, get_DCR, get_cm, get_class_cm, get_RD

In [6]:
INPUT_PATH="input"
OUTPUT_PATH="output"
CONFIG_PATH="config"

INPUT_DATA="test_data2"
INPUT_DATA_CONFIG="input_config"
OUTPUT_DATA="output"

INPUT_EXTENSION="csv"
INPUT_CONFIG_EXTENSION="json"
OUTPUT_EXTENSION="csv"

INPUT_FILE=f"{INPUT_DATA}.{INPUT_EXTENSION}"
INPUT_CONFIG_FILE=f"{INPUT_DATA_CONFIG}.{INPUT_CONFIG_EXTENSION}"
OUTPUT_FILE=f"{OUTPUT_DATA}.{OUTPUT_EXTENSION}"

INPUT_ABS_APTH=os.path.abspath(os.path.join(INPUT_PATH, INPUT_FILE))
INPUT_FILE_CONFIG=os.path.abspath(os.path.join(CONFIG_PATH, INPUT_CONFIG_FILE))
OUTPUT_ABS_APTH=os.path.abspath(os.path.join(OUTPUT_PATH, OUTPUT_FILE))

In [7]:
with open(INPUT_FILE_CONFIG, encoding='utf-8') as f:
    CONFIG = json.load(f)

In [8]:
CONFIG

{'INPUTS': {'FILE_NAME': ['test_data2.csv'],
  'SEPARATOR': ',',
  'DECIMAL': None,
  'ENCODING': 'utf-8',
  'FLOAT_PRECISION': 'high',
  'INDEXES': ['period_end_date', 'translated_when'],
  'DATE_COLUMNS': ['period_end_date', 'translated_when'],
  'DTYPE': {'if_data_corrected': 'object',
   'prod_gr_id': 'object',
   'country_id_n': 'object',
   'delivery_type_id': 'object',
   'freq_id': 'object',
   'retailer_id': 'object',
   'brand_id': 'object',
   'predict_automatch': 'float',
   'class_acctual': 'float'},
  'COLUMNS_WITH_NAN_VALUES': [None]},
 'MODEL': [None],
 'OUTPUTS': {'NAME': ['output.csv']}}

# Load data

In [9]:
# LOAD CSV
df = pd.read_csv(
    INPUT_ABS_APTH,
#     index_col=CONFIG['INPUTS']['INDEXES'],
    sep=CONFIG['INPUTS']['SEPARATOR'],
    # quotechar='"',
    # thousands=',',
#     decimal=CONFIG['INPUTS']['DECIMAL'],
    encoding=CONFIG['INPUTS']['ENCODING'],
    infer_datetime_format=True,
    parse_dates=CONFIG['INPUTS']['DATE_COLUMNS'],
    engine="c",
    low_memory=False,
#     float_precision=CONFIG['INPUTS']['DATE_COLUMNS'],
    skipinitialspace=True,
    dtype=CONFIG['INPUTS']['DTYPE']
)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19697 entries, 0 to 19696
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   period_end_date    19640 non-null  datetime64[ns, UTC]
 1   translated_when    19697 non-null  datetime64[ns, UTC]
 2   if_data_corrected  19697 non-null  object             
 3   prod_gr_id         19697 non-null  object             
 4   country_id_n       18405 non-null  object             
 5   delivery_type_id   18362 non-null  object             
 6   freq_id            19697 non-null  object             
 7   retailer_id        19697 non-null  object             
 8   brand_id           19697 non-null  object             
 9   predict_automatch  19368 non-null  float64            
 10  class_acctual      19697 non-null  float64            
dtypes: datetime64[ns, UTC](2), float64(2), object(7)
memory usage: 1.7+ MB


In [11]:
df.head(3)

,period_end_date,translated_when,if_data_corrected,prod_gr_id,country_id_n,delivery_type_id,freq_id,retailer_id,brand_id,predict_automatch,class_acctual
0,NaT,2020-10-15 06:58:28+00:00,0,426,121,NaN,2,96,111,1.0,1.0
1,NaT,2020-10-15 06:38:34+00:00,0,426,NaN,NaN,2,92,95,0.0,1.0
2,2020-09-20 00:00:00+00:00,2020-09-23 12:27:51+00:00,0,426,121,42730,2,30,37,1.0,1.0


In [12]:
df.describe(
    include='all',
    datetime_is_numeric=True
)

,period_end_date,translated_when,if_data_corrected,prod_gr_id,country_id_n,delivery_type_id,freq_id,retailer_id,brand_id,predict_automatch,class_acctual
count,19640,19697,19697,19697,18405,18362,19697,19697,19697,19368.000000,19697.000000
unique,NaN,NaN,2,3,34,914,2,52,199,NaN,NaN
top,NaN,NaN,0,426,121,31480,2,30,33,NaN,NaN
freq,NaN,NaN,17085,11844,4153,998,11934,1197,587,NaN,NaN
mean,2020-10-14 23:30:22.729124096+00:00,2020-10-22 06:01:21.525206784+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.636307,0.743210
min,2020-08-30 00:00:00+00:00,2020-09-01 03:05:51+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,2020-09-27 00:00:00+00:00,2020-09-29 13:45:29+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
50%,2020-10-18 00:00:00+00:00,2020-10-22 04:30:49+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000
75%,2020-11-08 00:00:00+00:00,2020-11-13 11:57:53+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000
max,2020-12-01 00:00:00+00:00,2021-02-01 14:50:49+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000


# Pandas data profiler

In [13]:
profile = ProfileReport(
    df,
    title="Pandas Profiling Report",
    pool_size=0,
    infer_dtypes=False
)

In [14]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

# Post-Processing Bias Metrics for the Trained Model

## Difference in positive proportion in predicted labels (DPPL)

![image info](./docs/images/DPPL.png)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)


### CHARACTERISTICS OF METRIC:
> DPPPL > -1 AND DPPPL < 1

> For example, if the model grants loans to 50% of class 2 and to 60% of class 1, then it may be biased against class 2.
We would have to decide whether a 10% difference is material.

## VARIABLE: prod_gr_id

### prod_gr_id total count

In [15]:
total_counts = df.groupby(['prod_gr_id']) \
    .size() \
    .rename('count_total') \
    .reset_index() \
    .sort_values(by='prod_gr_id', ascending=False) \
    .set_index('prod_gr_id')

# total_counts

### prod_gr_id by predicted classes

In [16]:
grouped_counts = df.groupby(['prod_gr_id', 'predict_automatch']) \
    .size() \
    .rename('count') \
    .reset_index() \
    .sort_values(by='prod_gr_id', ascending=False) \
    .set_index('prod_gr_id')

# grouped_counts

In [17]:
positive_grouped_counts = grouped_counts.loc[grouped_counts['predict_automatch'] == 1]

# positive_grouped_counts

In [18]:
merged_counts = pd.merge(positive_grouped_counts, total_counts, left_index=True, right_index=True)

# merged_counts

### (DPPL) for prod_gr_id

In [19]:
ppl_prod_gr_id = get_PPL(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count='predict_automatch'
)

ppl_prod_gr_id

prod_gr_id
427    0.663796
426    0.616177
413    0.622158
Name: PPL, dtype: float64

In [20]:
dppl_prod_gr_id = get_DPPL(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count='predict_automatch'
)

dppl_prod_gr_id

[-0.047618696554866724,
 -0.04163783945326527,
 -0.0059808571016014556,
 0.0059808571016014556,
 0.04163783945326527,
 0.047618696554866724]

#### Bucketizing dppl_prod_gr_id

In [21]:
print(pd.cut(dppl_prod_gr_id, bins=3).value_counts())

(-0.0477, -0.0159]    2
(-0.0159, 0.0159]     2
(0.0159, 0.0476]      2
dtype: int64


#### CONCLUSIONS:

> Basing solely on **DPPL** we can loosely assume that there is no disparity impact among **prod_gr_id** Classes

## VARIABLE: country_id_n

### country_id_n total count

In [22]:
total_counts = df.groupby(['country_id_n']) \
    .size() \
    .rename('count_total') \
    .reset_index() \
    .sort_values(by='country_id_n', ascending=False) \
    .set_index('country_id_n')

# total_counts

### country_id_n by predicted classes

In [23]:
grouped_counts = df.groupby(['country_id_n', 'predict_automatch']) \
    .size() \
    .rename('count') \
    .reset_index() \
    .sort_values(by='country_id_n', ascending=False) \
    .set_index('country_id_n')

# grouped_counts

In [24]:
positive_grouped_counts = grouped_counts.loc[grouped_counts['predict_automatch'] == 1]

# positive_grouped_counts

In [25]:
merged_counts = pd.merge(positive_grouped_counts, total_counts, left_index=True, right_index=True)

# merged_counts

### (DPPL) for country_id_n

In [26]:
ppl_country_id_n = get_PPL(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count='predict_automatch'
)

ppl_country_id_n

country_id_n
177     0.681818
176     0.664384
160     0.680851
139     0.568493
138     0.634615
136     0.689655
126     0.680945
121     0.649892
116     0.649402
114     0.688312
113     0.665432
110     0.649778
109     0.386667
108     0.690469
107     0.672269
106     0.430536
105     0.670455
104     0.699856
103     0.711691
1011    0.731844
1010    0.680288
1002    0.733333
Name: PPL, dtype: float64

In [27]:
dppl_country_id_n = get_DPPL(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count='predict_automatch'
)

dppl_country_id_n

[-0.3466666666666666,
 -0.3451769087523277,
 -0.32502459326522887,
 -0.3131890331890332,
 -0.30380193390713,
 -0.3029885057471265,
 -0.30279688216414485,
 -0.30164502164502166,
 -0.30130712424980594,
 -0.2951515151515151,
 -0.29427868045297884,
 -0.29418439716312056,
 -0.2936217948717949,
 -0.2856022408963586,
 -0.28378787878787876,
 -0.2811548087627071,
 -0.2787654320987654,
 -0.27771689497716895,
 -0.2693192486865114,
 -0.2632249779276025,
 -0.26311111111111113,
 -0.26273572377158033,
 -0.25993214940460824,
 -0.2591187212446047,
 -0.2577752371424999,
 -0.25128173064899334,
 -0.25040889595045707,
 -0.2503146126605988,
 -0.24975201036927314,
 -0.24794871794871792,
 -0.24173245639383684,
 -0.239918094285357,
 -0.23489564759624365,
 -0.23384711047464718,
 -0.21935519342508075,
 -0.21924132660858936,
 -0.21886593926905856,
 -0.20407893344619615,
 -0.1818264840182649,
 -0.16484018264840172,
 -0.1633504247340628,
 -0.14319810924696397,
 -0.13795669951574313,
 -0.13136254917076828,
 -0.12197

#### Bucketizing dppl_country_id_n

In [28]:
print(pd.cut(dppl_country_id_n, bins=3).value_counts())

(-0.347, -0.116]     47
(-0.116, 0.116]     368
(0.116, 0.347]       47
dtype: int64


#### CONCLUSIONS:

> Although it is likely. Basing solely on **DPPL** we can't assume if there is a disparity impact among **prod_gr_id** Classes yet. 

## Disparate (Adverse) Impact (DI)

![image info](./docs/images/DI.png)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)

### CHARACTERISTICS OF METRIC:
> **DI >= 0**

> This measure may be considered fair if it resides in the **<0.8; 1.2>** range.

## (DI) for Variable: prod_gr_id

In [29]:
ppl_prod_gr_id

prod_gr_id
427    0.663796
426    0.616177
413    0.622158
Name: PPL, dtype: float64

In [30]:
di_prod_gr_id = sorted([abs(i / j) for i in ppl_prod_gr_id for j in ppl_prod_gr_id if i != j])

#### Bucketizing di_prod_gr_id

In [31]:
print(pd.cut(di_prod_gr_id, bins=4).value_counts())

(0.928, 0.966]    2
(0.966, 1.003]    1
(1.003, 1.04]     1
(1.04, 1.077]     2
dtype: int64


#### CONCLUSIONS:

> For each possible relation between Classes the metric resides between **<0.8; 1.2>**

> We can loosely assume that there is no disparity impact between Classes of **prod_gr_id** Variable

## (DI) for Variable: country_id_n

In [32]:
ppl_country_id_n

country_id_n
177     0.681818
176     0.664384
160     0.680851
139     0.568493
138     0.634615
136     0.689655
126     0.680945
121     0.649892
116     0.649402
114     0.688312
113     0.665432
110     0.649778
109     0.386667
108     0.690469
107     0.672269
106     0.430536
105     0.670455
104     0.699856
103     0.711691
1011    0.731844
1010    0.680288
1002    0.733333
Name: PPL, dtype: float64

In [33]:
di_country_id_n = sorted([abs(i / j) for i in ppl_country_id_n for j in ppl_country_id_n if i != j])

#### Bucketizing di_country_id_n

In [34]:
print(pd.cut(di_country_id_n, bins=4).value_counts())

(0.526, 0.87]      57
(0.87, 1.212]     359
(1.212, 1.554]     14
(1.554, 1.897]     32
dtype: int64


In [35]:
pd.Series(di_country_id_n).describe()

count    462.000000
mean       1.025311
std        0.238095
min        0.527273
25%        0.943997
50%        1.000000
75%        1.059325
max        1.896552
dtype: float64

#### CONCLUSIONS:

> There is more than 100 Classes of Variable **country_id_n** out of **<0.8; 1.2>** bounds.

> At this moment we can loosely assume that there are some examples of Disparity Impact in case of **country_id_n** Variable.

## Difference in Conditional Outcome (DCO)
> Type 1: **Difference in Conditional Acceptance (DCA)**

> Type 2: **Difference in Conditional Rejection (DCR)**

> When both **DCA** and **DCR** are very close to **0**, we can conclude that the proportion of qualified (as suggested by observed labels) applicants accepted by the model and the proportion of unqualified applicants rejected are nearly equal across both classes.

### Difference in Conditional Acceptance (DCA)

![image info](./docs/images/DCA.jpg)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)

#### CHARACTERISTICS OF METRIC:
> Unbounded

> Zero denominator is possible. In such case the allocations to each Class are too small and a warning should be issued.

## (DCA) for Variable: prod_gr_id

In [36]:
ca_prod_gr_id = get_CA(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

ca_prod_gr_id

prod_gr_id
427    1.114989
426    1.203069
413    1.206378
Name: CA, dtype: float64

In [37]:
dca_prod_gr_id = get_DCA(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

dca_prod_gr_id

[-0.09138882810439752,
 -0.08808051974645381,
 -0.0033083083579437123,
 0.0033083083579437123,
 0.08808051974645381,
 0.09138882810439752]

#### Bucketizing dca_prod_gr_id

In [38]:
print(pd.cut(dca_prod_gr_id, bins=3).value_counts())

(-0.0916, -0.0305]    2
(-0.0305, 0.0305]     2
(0.0305, 0.0914]      2
dtype: int64


#### CONCLUSIONS:

> TODO: Diffrances are really small

## (DCA) for Variable: country_id_n

In [39]:
ca_country_id_n = get_CA(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

ca_country_id_n

country_id_n
177     1.040000
176     1.120766
160     1.031250
139     1.317269
138     1.212121
136     1.050000
126     1.090022
121     1.133012
116     1.147239
114     1.085954
113     1.123377
110     1.136799
109     2.000000
108     1.072022
107     1.079167
106     1.686901
105     1.105932
104     1.045361
103     1.063796
1011    1.091603
1010    1.116608
1002    1.090909
Name: CA, dtype: float64

In [40]:
dca_country_id_n = get_DCA(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

dca_country_id_n

[-0.96875,
 -0.96,
 -0.9546391752577319,
 -0.95,
 -0.9362041467304625,
 -0.9279778393351801,
 -0.9208333333333334,
 -0.9140461215932913,
 -0.9099783080260304,
 -0.9090909090909092,
 -0.9083969465648856,
 -0.8940677966101696,
 -0.88339222614841,
 -0.8792341678939617,
 -0.8766233766233766,
 -0.8669877732493516,
 -0.8632010943912449,
 -0.852760736196319,
 -0.7878787878787878,
 -0.6827309236947792,
 -0.6556509584664536,
 -0.6469009584664536,
 -0.6415401337241855,
 -0.6369009584664536,
 -0.6231051051969161,
 -0.6148787978016337,
 -0.607734291799787,
 -0.600947080059745,
 -0.596879266492484,
 -0.5959918675573628,
 -0.5952979050313392,
 -0.5809687550766232,
 -0.5702931846148636,
 -0.5661351263604153,
 -0.5635243350898302,
 -0.5538887317158052,
 -0.5501020528576985,
 -0.5396616946627726,
 -0.47477974634524145,
 -0.36963188216123277,
 -0.3130990415335464,
 -0.28601907630522083,
 -0.2772690763052208,
 -0.27190825156295273,
 -0.2672690763052208,
 -0.2534732230356833,
 -0.2452469156404009,
 -0.238

#### Bucketizing dca_prod_gr_id

In [41]:
print(pd.cut(dca_country_id_n, bins=3).value_counts())

(-0.971, -0.323]     40
(-0.323, 0.323]     382
(0.323, 0.969]       40
dtype: int64


#### CONCLUSIONS:

> TODO:

### Difference in Conditional Rejection (DCR)

![image info](./docs/images/DCR.jpg)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)

#### CHARACTERISTICS OF METRIC:
> Unbounded

> Zero denominator is possible. In such case the allocations to each Class are too small and a warning should be issued.

## (DCR) for Variable: prod_gr_id

In [42]:
cr_prod_gr_id = get_CR(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

cr_prod_gr_id

prod_gr_id
427    0.784050
426    0.716223
413    0.678182
Name: CR, dtype: float64

In [43]:
dcr_prod_gr_id = get_DCR(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

dcr_prod_gr_id

[-0.10586836102965136,
 -0.06782764531712637,
 -0.038040715712524986,
 0.038040715712524986,
 0.06782764531712637,
 0.10586836102965136]

#### Bucketizing dca_prod_gr_id

In [44]:
print(pd.cut(dcr_prod_gr_id, bins=3).value_counts())

(-0.106, -0.0353]    3
(-0.0353, 0.0353]    0
(0.0353, 0.106]      3
dtype: int64


#### CONCLUSIONS:

> TODO:

## (DCR) for Variable: country_id_n

In [45]:
cr_country_id_n = get_CR(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

cr_country_id_n

country_id_n
177     0.914286
176     0.765396
160     0.933333
139     0.582011
138     0.648649
136     0.888889
126     0.817330
121     0.784384
116     0.920863
114     0.817757
113     0.768797
110     0.942308
109     0.372807
108     0.849844
107     0.844828
106     0.481840
105     0.798246
104     0.916256
103     0.856000
1011    0.765957
1010    0.763359
1002    0.750000
Name: CR, dtype: float64

In [46]:
dcr_country_id_n = get_DCR(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

dcr_country_id_n

[-0.5695006747638327,
 -0.5605263157894738,
 -0.5480562918086584,
 -0.5434491400916084,
 -0.5414786967418546,
 -0.5160818713450293,
 -0.48319298245614034,
 -0.47703656952600476,
 -0.47202056866303693,
 -0.46046749860309183,
 -0.4514931396287329,
 -0.4449499918019348,
 -0.4445231932289741,
 -0.4390231156479176,
 -0.4344159639308675,
 -0.4324455205811138,
 -0.4254385964912281,
 -0.4115769366108682,
 -0.4070486951842884,
 -0.3959899749373434,
 -0.393150429264651,
 -0.39258887688429284,
 -0.3905517610820946,
 -0.37719298245614036,
 -0.3741598062953995,
 -0.36800339336526394,
 -0.3629873925022961,
 -0.3602971102971103,
 -0.35132275132275137,
 -0.33885272734193606,
 -0.33591681564119397,
 -0.3354900170682333,
 -0.334245575624886,
 -0.3322751322751323,
 -0.3164054203304873,
 -0.30687830687830686,
 -0.3025437604501274,
 -0.2936590436590436,
 -0.2869567987766026,
 -0.28468468468468466,
 -0.2841172531039102,
 -0.283555700723552,
 -0.2815185849213538,
 -0.27584163110478904,
 -0.273989417989418,
 

#### Bucketizing dca_prod_gr_id

In [47]:
print(pd.cut(dcr_country_id_n, bins=3).value_counts())

(-0.571, -0.19]     61
(-0.19, 0.19]      340
(0.19, 0.57]        61
dtype: int64


#### CONCLUSIONS:

> TODO:

In [48]:
cm_by_class_prod_gr_id = get_class_cm(
    data_frame=df,
    acctuals='class_acctual',
    predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

cm_by_class_prod_gr_id

,Class,TN,FN,FP,TP
0,426,75,193,2989,8587
1,427,4,12,871,2480
2,413,11,34,1108,3333


In [49]:
rd_prod_gr_id = get_RD(
    data_frame=df,
    acctuals='class_acctual',
    predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

rd_prod_gr_id

[-0.01716636745558453,
 -0.01188376666736568,
 -0.00528260078821885,
 0.00528260078821885,
 0.01188376666736568,
 0.01716636745558453]

#### Bucketizing dca_prod_gr_id

In [50]:
print(pd.cut(rd_prod_gr_id, bins=3).value_counts())

(-0.0172, -0.00572]    2
(-0.00572, 0.00572]    2
(0.00572, 0.0172]      2
dtype: int64


#### CONCLUSIONS:

> TODO:

## (RD) for Variable: country_id_n

In [51]:
cm_by_class_prod_gr_id = get_class_cm(
    data_frame=df,
    acctuals='class_acctual',
    predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

cm_by_class_prod_gr_id

,Class,TN,FN,FP,TP
0,121,17,41,1078,3017
1,108,3,9,812,2313
2,176,0,2,261,759
3,110,25,57,269,774
4,139,0,0,110,328
5,114,1,1,174,517
6,105,0,2,91,259
7,116,11,26,117,348
8,106,0,1,199,527
9,104,3,2,183,505


In [52]:
recall_by_class = get_RD(
    data_frame=df,
    acctuals='class_acctual',
    predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

recall_by_class

[-1.0,
 -0.9981060606060606,
 -0.9980694980694981,
 -0.9973718791064389,
 -0.9970014992503748,
 -0.9961389961389961,
 -0.9961240310077519,
 -0.9960552268244576,
 -0.9960199004975124,
 -0.9936708860759493,
 -0.993103448275862,
 -0.993006993006993,
 -0.9925681255161024,
 -0.9923371647509579,
 -0.986592544146501,
 -0.9314079422382672,
 -0.93048128342246,
 -0.06951871657754005,
 -0.06859205776173283,
 -0.0676247771836006,
 -0.06758821464703813,
 -0.06689059568397893,
 -0.06669811836779338,
 -0.0666615558312309,
 -0.0665202158279149,
 -0.0659639368681717,
 -0.0656577127165362,
 -0.06564274758529198,
 -0.06559355701210767,
 -0.06557394340199763,
 -0.06553861707505249,
 -0.06473105390072897,
 -0.06471608876948476,
 -0.0646472845861904,
 -0.06461195825924526,
 -0.06318960265348939,
 -0.0626221648534021,
 -0.06252570958453307,
 -0.06226294383768216,
 -0.06208684209364246,
 -0.06185588132849795,
 -0.06169550603759488,
 -0.06159905076872585,
 -0.061160183277835234,
 -0.06092922251269073,
 -0.0561

#### Bucketizing dca_prod_gr_id

In [53]:
print(pd.cut(recall_by_class, bins=3).value_counts())

(-1.002, -0.333]     17
(-0.333, 0.333]     270
(0.333, 1.0]         17
dtype: int64


#### CONCLUSIONS:

> TODO:

## Difference in label rates (DLR)

![image info](./docs/images/DAR.png)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)

#### CHARACTERISTICS OF METRIC:
> DAR > -1 AND DAR < 1

> DAR is the same as precision difference between the first and second classes.

## (DAR) for Variable: prod_gr_id

In [107]:
def get_AR(data_frame, column_to_count_acctuals, column_to_count_predictions, column_to_group_by):
    
    counts_total_labels = data_frame.groupby([column_to_group_by]) \
    .size() \
    .rename('count_total') \
    .reset_index() \
    .sort_values(by=column_to_group_by, ascending=False) \
    .set_index(column_to_group_by)
    
    
    counts_grouped_acctuals = data_frame.groupby([column_to_group_by, column_to_count_acctuals]) \
    .size() \
    .rename('count_grouped_acctuals') \
    .reset_index() \
    .sort_values(by=column_to_group_by, ascending=False) \
    .set_index(column_to_group_by)
    
    positive_counts_grouped_acctuals = counts_grouped_acctuals.loc[counts_grouped_acctuals[column_to_count_acctuals] == 1]
    
    
    counts_grouped_predictions = data_frame.groupby([column_to_group_by, column_to_count_predictions]) \
    .size() \
    .rename('count_gruped_predictions') \
    .reset_index() \
    .sort_values(by=column_to_group_by, ascending=False) \
    .set_index(column_to_group_by)

    positive_counts_grouped_predictions= counts_grouped_predictions.loc[counts_grouped_predictions[column_to_count_predictions] == 1]


    counts = pd.merge(positive_counts_grouped_acctuals, positive_counts_grouped_predictions, left_index=True, right_index=True)
#     print(counts)
    
    data = pd.DataFrame(columns=['Class', 'TN', 'FN', 'FP', 'TP'])
    
    for group in data_frame[column_to_group_by].dropna().unique():
        sample = data_frame.loc[data_frame[column_to_group_by] == group]
        cm = confusion_matrix(sample[column_to_count_acctuals], sample[column_to_count_predictions].notnull())
        try:
            TN, FN, FP, TP = cm[0][0], cm[1][0], cm[0][1], cm[1][1]
        except:
            pass
        else:
            TN, FN, FP, TP = cm[0][0], cm[1][0], cm[0][1], cm[1][1]
        
        data = data.append({'Class': group, 'TN': TN, 'FN': FN, 'FP': FP, 'TP': TP}, ignore_index=True)
    data.set_index('Class')
#     print(data)
                
    output = counts.merge(data, left_on='prod_gr_id', right_on='Class')
    
    return output

In [108]:
ar_by_class_prod_gr_id = get_AR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

test

,class_acctual,count_grouped_acctuals,predict_automatch,count_gruped_predictions,Class,TN,FN,FP,TP
0,1.0,78,1.0,75,177,0,0,32,78
1,1.0,761,1.0,679,176,0,2,261,759
2,1.0,33,1.0,32,160,0,0,14,33
3,1.0,328,1.0,249,139,0,0,110,328
4,1.0,80,1.0,66,138,1,0,23,80
5,1.0,21,1.0,20,136,0,0,8,21
6,1.0,1005,1.0,922,126,1,4,348,1001
7,1.0,3058,1.0,2699,121,17,41,1078,3017
8,1.0,374,1.0,326,116,11,26,117,348
9,1.0,518,1.0,477,114,1,1,174,517


In [109]:
def get_DAR(data_frame, column_to_count_acctuals, column_to_count_predictions, column_to_group_by):
    
    ar = []
    
    data = get_AR(data_frame, column_to_count_acctuals, column_to_count_predictions, column_to_group_by)
    
    data['AR'] = data['TP'] / data['count_gruped_predictions']
           
    for i in data['AR'].iteritems():
        ar.append(i[1])
    
    return sorted(set([i - j for i in ar for j in ar if i != j]))

In [105]:
test2 = get_DAR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

test2

[-0.0845759420059844,
 -0.06700404572856522,
 -0.017571896277419174,
 0.017571896277419174,
 0.06700404572856522,
 0.0845759420059844]